# 🔍 S3 Diagnostic Testing Notebook
Test S3 connectivity, permissions, and upload functionality

In [ ]:
# Install required packages
!pip install boto3 python-dotenv -q

In [ ]:
import boto3
import os
import pickle
import tempfile
from datetime import datetime

# Load credentials
try:
    from google.colab import userdata
    AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
    AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')
    print("✅ Loaded credentials from Google Colab secrets")
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
    AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
    AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
    print("✅ Loaded credentials from .env file")

# Configuration
S3_BUCKET = "byupathway-public"
CACHE_PREFIX = "embeddings-cache"
TEST_PREFIX = "test-diagnostics"

print(f"\n📦 Target Bucket: {S3_BUCKET}")
print(f"🗂️  Cache Prefix: {CACHE_PREFIX}")
print(f"🧪 Test Prefix: {TEST_PREFIX}")

## Test 1: AWS Credentials Validation

In [ ]:
print("\n" + "="*60)
print("TEST 1: AWS Credentials Validation")
print("="*60)

try:
    # Create boto3 client
    s3 = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name='us-east-1'
    )
    
    # Test credentials by listing buckets
    response = s3.list_buckets()
    print("✅ AWS Credentials Valid")
    print(f"✅ Found {len(response['Buckets'])} accessible buckets")
    
    # Check if target bucket exists
    bucket_names = [b['Name'] for b in response['Buckets']]
    if S3_BUCKET in bucket_names:
        print(f"✅ Target bucket '{S3_BUCKET}' is accessible")
    else:
        print(f"⚠️  Target bucket '{S3_BUCKET}' not in your accessible buckets list")
        print(f"   Your accessible buckets: {bucket_names}")
        
except Exception as e:
    print(f"❌ Credentials Test Failed: {str(e)}")
    print(f"   Error Type: {type(e).__name__}")

## Test 2: Bucket Access & List Permissions

In [ ]:
print("\n" + "="*60)
print("TEST 2: Bucket Access & List Permissions")
print("="*60)

try:
    # Try to list objects in the cache prefix
    response = s3.list_objects_v2(
        Bucket=S3_BUCKET,
        Prefix=CACHE_PREFIX,
        MaxKeys=10
    )
    
    object_count = response.get('KeyCount', 0)
    print(f"✅ Successfully listed objects in '{S3_BUCKET}/{CACHE_PREFIX}'")
    print(f"✅ Found {object_count} objects (showing max 10)")
    
    if object_count > 0:
        print("\nSample objects:")
        for obj in response.get('Contents', [])[:3]:
            print(f"  - {obj['Key']} ({obj['Size']} bytes)")
    
except Exception as e:
    print(f"❌ List Objects Failed: {str(e)}")
    print(f"   Error Type: {type(e).__name__}")
    if hasattr(e, 'response'):
        print(f"   HTTP Status: {e.response.get('ResponseMetadata', {}).get('HTTPStatusCode')}")
        print(f"   Error Code: {e.response.get('Error', {}).get('Code')}")

## Test 3: Write/Upload Permissions (Test Prefix)

In [ ]:
print("\n" + "="*60)
print("TEST 3: Write/Upload Permissions (Test Prefix)")
print("="*60)

try:
    # Create a small test file
    test_data = {
        'timestamp': datetime.now().isoformat(),
        'test': 'S3 upload diagnostic',
        'values': [1, 2, 3, 4, 5]
    }
    
    # Save to temp file
    with tempfile.NamedTemporaryFile(mode='wb', suffix='.pkl', delete=False) as tmp:
        pickle.dump(test_data, tmp)
        tmp_path = tmp.name
    
    # Generate test key
    test_key = f"{TEST_PREFIX}/diagnostic_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
    
    print(f"📤 Attempting upload to: s3://{S3_BUCKET}/{test_key}")
    
    # Upload to S3
    s3.upload_file(
        tmp_path,
        S3_BUCKET,
        test_key
    )
    
    print(f"✅ Upload successful!")
    
    # Verify the upload
    response = s3.head_object(Bucket=S3_BUCKET, Key=test_key)
    print(f"✅ File verified on S3")
    print(f"   Size: {response['ContentLength']} bytes")
    print(f"   Last Modified: {response['LastModified']}")
    
    # Clean up local file
    os.unlink(tmp_path)
    
except Exception as e:
    print(f"❌ Upload Test Failed: {str(e)}")
    print(f"   Error Type: {type(e).__name__}")
    if hasattr(e, 'response'):
        print(f"   HTTP Status: {e.response.get('ResponseMetadata', {}).get('HTTPStatusCode')}")
        print(f"   Error Code: {e.response.get('Error', {}).get('Code')}")
        print(f"   Error Message: {e.response.get('Error', {}).get('Message')}")
    
    # Clean up if file exists
    if 'tmp_path' in locals() and os.path.exists(tmp_path):
        os.unlink(tmp_path)

## Test 4: Write to Cache Prefix (Actual Location)

In [ ]:
print("\n" + "="*60)
print("TEST 4: Write to Cache Prefix (Actual Location)")
print("="*60)

try:
    # Create a small test file
    test_data = {
        'timestamp': datetime.now().isoformat(),
        'test': 'Cache prefix upload test',
        'embedding': [0.1, 0.2, 0.3, 0.4, 0.5]
    }
    
    # Save to temp file (simulating the actual cache save)
    with tempfile.NamedTemporaryFile(mode='wb', suffix='.pkl', delete=False) as tmp:
        pickle.dump(test_data, tmp)
        tmp_path = tmp.name
    
    # Generate cache key (similar to actual implementation)
    cache_key = f"{CACHE_PREFIX}/diagnostic_cache_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
    
    print(f"📤 Attempting upload to: s3://{S3_BUCKET}/{cache_key}")
    
    # Upload to S3
    s3.upload_file(
        tmp_path,
        S3_BUCKET,
        cache_key
    )
    
    print(f"✅ Cache upload successful!")
    
    # Verify the upload
    response = s3.head_object(Bucket=S3_BUCKET, Key=cache_key)
    print(f"✅ File verified on S3")
    print(f"   Size: {response['ContentLength']} bytes")
    print(f"   ETag: {response['ETag']}")
    
    # Clean up local file
    os.unlink(tmp_path)
    
except Exception as e:
    print(f"❌ Cache Upload Test Failed: {str(e)}")
    print(f"   Error Type: {type(e).__name__}")
    if hasattr(e, 'response'):
        print(f"   HTTP Status: {e.response.get('ResponseMetadata', {}).get('HTTPStatusCode')}")
        print(f"   Error Code: {e.response.get('Error', {}).get('Code')}")
        print(f"   Error Message: {e.response.get('Error', {}).get('Message')}")
    
    # Clean up if file exists
    if 'tmp_path' in locals() and os.path.exists(tmp_path):
        os.unlink(tmp_path)

## Test 5: Network Connectivity & Region Check

In [ ]:
print("\n" + "="*60)
print("TEST 5: Network Connectivity & Region Check")
print("="*60)

try:
    # Get bucket location
    response = s3.get_bucket_location(Bucket=S3_BUCKET)
    bucket_region = response['LocationConstraint'] or 'us-east-1'
    print(f"✅ Bucket Region: {bucket_region}")
    
    # Check if we're using the right region
    client_region = s3.meta.region_name
    print(f"✅ Client Region: {client_region}")
    
    if bucket_region == client_region or (bucket_region == 'us-east-1' and client_region == 'us-east-1'):
        print("✅ Region configuration is correct")
    else:
        print(f"⚠️  Region mismatch! Bucket is in {bucket_region} but client is configured for {client_region}")
    
    # Test basic connectivity
    import socket
    hostname = f"s3.{bucket_region}.amazonaws.com" if bucket_region != 'us-east-1' else "s3.amazonaws.com"
    print(f"\n🌐 Testing connectivity to {hostname}...")
    socket.create_connection((hostname, 443), timeout=5)
    print("✅ Network connectivity to S3 is working")
    
except socket.timeout:
    print("❌ Network timeout - unable to reach S3 endpoint")
except socket.error as e:
    print(f"❌ Network error: {str(e)}")
except Exception as e:
    print(f"❌ Region Check Failed: {str(e)}")
    print(f"   Error Type: {type(e).__name__}")

## Test 6: IAM Permissions Summary

In [ ]:
print("\n" + "="*60)
print("TEST 6: IAM Permissions Summary")
print("="*60)

try:
    # Try to get caller identity
    sts = boto3.client(
        'sts',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name='us-east-1'
    )
    
    identity = sts.get_caller_identity()
    print(f"✅ AWS Account: {identity['Account']}")
    print(f"✅ User ARN: {identity['Arn']}")
    print(f"✅ User ID: {identity['UserId']}")
    
    print("\n📋 Required Permissions for this workflow:")
    permissions_needed = [
        "s3:ListBucket (on bucket)",
        "s3:GetObject (on objects)",
        "s3:PutObject (on objects)",
        "s3:PutObjectAcl (for public-read ACL)",
        "s3:DeleteObject (for cache cleanup)"
    ]
    
    for perm in permissions_needed:
        print(f"  • {perm}")
    
except Exception as e:
    print(f"⚠️  Could not retrieve IAM identity: {str(e)}")

## 📊 Diagnostic Summary

In [ ]:
print("\n" + "="*60)
print("📊 DIAGNOSTIC SUMMARY")
print("="*60)
print("\nIf all tests passed, the issue might be:")
print("  1. Race condition with many concurrent uploads")
print("  2. Temporary network instability")
print("  3. Rate limiting from too many rapid requests")
print("\nIf Test 3 or 4 failed, the issue is likely:")
print("  1. Missing s3:PutObject permission")
print("  2. Bucket policy blocking uploads")
print("  3. Wrong bucket name or region")
print("\nIf Test 1 failed:")
print("  1. Invalid AWS credentials")
print("  2. Credentials not properly set in Colab secrets")
print("="*60)